In [1]:
# Testing Cell

# FLOPS Based Blended Wing Body Modeling

The blended wing body aircraft is modeled following FLOPS implementation. It is important to point out the the Aviary implementation is limited to certain assumptions. To model a special model, the user must read this document carefully to see it is within the scope of the Aviary implementation.

## FLOPS Based Geometry

The FLOPS based geometry subsystems are shown in the following table. The left column is for the conventional aircraft and the right column is for BWB aircraft. In both case, user can choose a simple layout or a detailed layout.

**Comparision of Conventional Aircraft and BWB Aircraft**

| Tube+Wing | BWB |
| --------- | ----- |
| `SimpleCabinLayout` or `DetailedCabinLayout`  | `BWBSimpleCabinLayout` or `BWBDetailedCabinLayout` |
|   | `BWBUpdateDetailedWingDist` or `BWBComputeDetailedWingDist` |
| `FuselagePrelim` | `BWBFuselagePrelim` |
| `WingPrelim` | `BWBWingPrelim` |
| `_Prelim` | `_Prelim` |
| `_Wing` | `_BWBWing` |
| `_Tail` | `_Tail` |
| `_Fuselage` | `_BWBFuselage` |
| `_FuselageRatios` | `_FuselageRatios` |
| `Nacelles` | `Nacelles` |
| `Canard` | `Canard` |
| `WingCharacteristicLength` | `BWBWingCharacteristicLength` |
| `OtherCharacteristicLengths` | `OtherCharacteristicLengths` |
| `TotalWettedArea` | `TotalWettedArea` |

As we see in the above table, some geometrical components have no special treatments in the BWB case (e.g. `Nacelles` and `_Tail`). So, the new implementation of BWB geometry uses the same components in conventional aircraft.

Comparing to traditional tube and wing model, Blended wing body (BWB) modeling has two major new changes in geometry subsystems:

- Wings always have detailed wing data,
- Fuselage layout is computed using a special algorithm.

We will explain some details of each feature in this document.

In [ ]:
# Testing Cell

from aviary.api import Aircraft
from aviary.utils.doctape import get_variable_name, glue_variable

# new variables added for FLOPS based BWB geometry
glue_variable(
    get_variable_name(Aircraft.CrewPayload.Design.NUM_SEATS_ABREAST_BUSINESS), md_code=True
)
glue_variable(get_variable_name(Aircraft.CrewPayload.Design.NUM_SEATS_ABREAST_FIRST), md_code=True)
glue_variable(
    get_variable_name(Aircraft.CrewPayload.Design.NUM_SEATS_ABREAST_TOURIST), md_code=True
)
glue_variable(get_variable_name(Aircraft.CrewPayload.Design.SEAT_PITCH_BUSINESS), md_code=True)
glue_variable(get_variable_name(Aircraft.CrewPayload.Design.SEAT_PITCH_FIRST), md_code=True)
glue_variable(get_variable_name(Aircraft.CrewPayload.Design.SEAT_PITCH_TOURIST), md_code=True)

`Aircraft.CrewPayload.Design.NUM_SEATS_ABREAST_BUSINESS`

`Aircraft.CrewPayload.Design.NUM_SEATS_ABREAST_FIRST`

`Aircraft.CrewPayload.Design.NUM_SEATS_ABREAST_TOURIST`

`Aircraft.CrewPayload.Design.SEAT_PITCH_BUSINESS`

`Aircraft.CrewPayload.Design.SEAT_PITCH_FIRST`

`Aircraft.CrewPayload.Design.SEAT_PITCH_TOURIST`

### Fuselage Layout

There are two options: simple layout and detailed layout.



### Detailed Wings

For BWB, there are always detailed wings. They are either given (BWBUpdateDetailedWingDist) or created (BWBComputeDetailedWingDist). This means that number of input wing stations is always positive. If input wing stations are not provided, FLOPS adds three points. We call this case the simple wing case. Otherwise, we say that we have a detailed wing case.